In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import string
import pickle as pkl
import numpy as np

TRUE_CUTOFF = 3150

In [2]:
replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
red_flag_char = [" AVE ", " ST ", " AV ", " BLVD ", " BLV ", " RD ", " DR "]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

In [3]:
# remake the dict
map_dict = {}

# use previous dict
# map_dict = pkl.load(open('trueheaders_match_dict.pkl', 'rb'))

In [4]:
def clean_header(h):
    red_flag = False
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = ''.join(hl).upper()
    for rf in red_flag_char:
        if rf in h: red_flag = True
    if cnt > 3 or red_flag: h = ""
    return h
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the ocr header and the true header
    return fuzz.ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
            score_tuple = sorted_score[0]
            if score_tuple[0] > 90:
                print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], score_tuple[1], "TRUE")
            else:
                print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], "no_header", "FALSE")
    return map_dict

In [5]:
raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

In [6]:
df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True).sort_values("count")
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df[TRUE_CUTOFF:].true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

LOG: Matched! PLUMBERS' SUPPLIES and PLUMBERSL SUPPLIES with score 94
LOG: Matched! ROVING CANS AND SMINGS and ROVING CANS AND SPRINGS with score 93
LOG: Matched! APROVLSION DEALERS  PUBLIC CONVEYANCES and PROVISION DEALERS  F'PUBLIC CONVEVANCES with score 91
LOG: Matched! INDUSTRIAL SCRAP METAL and LNDUSTRIAL SCRAP METAL with score 95
LOG: Not matched, DOORS-GARAAE and MAFKERS-GRAVE with score 56
LOG: Not matched, WARNIBH DEALERS and HAFRNSS DEALERS with score 73
LOG: Not matched, WATER HEATERS-DEALERS and WATER HEATERS-GAS with score 84
LOG: Matched! WASHING COMPOUNDS-MANU- FACTURERS  and WASHING COMPOUNDS-MANU- FAC TURERS with score 97
LOG: Not matched, HOSIERY-WHOLESALE DISTRIBU- TORS and  HOSIE RY-WHOLESALE AND DISTRIBUTORS with score 88
LOG: Not matched,  ARTISTS' PAINTS and ARTISTS' SMOCKS with score 65
LOG: Matched! REAL ESTATE MANAGEMENT and REAL HTATE MANAGEMENT with score 93
LOG: Matched! LWOMEWS  MISSES' COATS  DRESSES and WOMEN'S  MISSES' COATS  DRESSES with score 94
LOG: 

LOG: Not matched, ARTISTS' PAINTS and ARTISTS' SMOCKS with score 67
LOG: Not matched, CORSETIER and CORSETS with score 75
LOG: Not matched, RADIO AND TELEVISION BROAD- . CASTING COMPANIES AND STATIONS and RADIO AND TELEVISION SETSNAND SUPPLILB-WHOLI'SALE with score 59
LOG: Matched! GLASS PINS  RINGS and OLASS PINS  RINGS with score 94
LOG: Not matched, A PEARL MFRS and TABLE PAD MFRS with score 69
LOG: Not matched, N OTARIES-PUBLIC and N AT ARIES-PUBLLC with score 85
LOG: Not matched, CLOTH'FNE MANUFACTURERS- CHILDREN2S and CLOTHING MANUFAC TURERS- CHILDREN3S with score 89
LOG: Matched! BROOMS, BRUSHES 'MOPS-MTRS and BROOMS, BRUSHES  MOPS-MTRS with score 96
LOG: Not matched, COSTUMES-THEATRICAL- MASQUERADE and COMMERCIAL PAPER with score 51
LOG: Matched! LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIO and LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIMI with score 97
LOG: Matched! EXTERMINATORS-BUG AND VERM- I and EXTERMINATORS-BUG AND VERMI with score 96
LOG: Not matched, TWLSTER TRAVEL

LOG: Not matched, INSURANCE COUNSE LIDRS and INSURANCE -BONDS with score 74
LOG: Not matched, AWNLNGS-L'OR WEDDINGS  and SEALING-POR MAILING with score 54
LOG: Not matched, CEILINGS and CASTINGS  with score 59
LOG: Not matched, REGULATOR MANUFACTURERS- TEMPERATURE AND PRESSURE and REGULATOR MFRS TEMPERATURE AND PRESSURE with score 89
LOG: Not matched, LUBRICATING OILS  IMGGAGE and TLUBRICATING OILS with score 76
LOG: Not matched, LADDERS ' STAGING and BLANKETS  LANPINGS with score 57
LOG: Matched! CHECKING ACCOUNTS-THRIITI- CHECK SERVICE and GHECKING ACCOUNTS-THRIITI- CHECK SERVICE with score 98
LOG: Not matched,  HOME FURNISHERS and HOME PIUNISHERIS with score 81
LOG: Not matched, GONTRACTORS-TLLLNG and CONTRACTORS-TLLING with score 89
LOG: Matched!  PREFABRICATED PIPING and WREFABRICATED PIPING with score 93
LOG: Not matched, INSURANOE-AUTOTMOHLLE and INSURANEE-AUTOMOBILE with score 83
LOG: Matched! ADVERTISING REPRESENTATIVES- NEWSPAPER AND'MAGAZ INE and ADVERTISING REPRESENTATIVES 

LOG: Not matched, SAWDUST FOR BEDDING CATTLE and SACHETS FOR GREETING; CARDS with score 57
LOG: Matched! DRIVEWAYS AND SIDEWALKS- CONSTRUCTIO and DRIVEWAYS  SIDEWALKS-CON- STRUCTIO with score 91
LOG: Not matched, WOOD ENGRAVERS  WOOD PRODUCTS-MIRS and WAX  WAX PRODUCTS MFRS with score 61
LOG: Not matched, LUGGAQE--RETAIL and LUGGAGE-RETAIL with score 90
LOG: Matched! RUBBER GOODS-RETALL  WHOLE- SALE and RUBBER GOODS-RETAIL  WHOLE- SALE with score 97
LOG: Matched! FFPAVING CONTRACTORS--ASPHALT  CONCRETE and PAVING CONTRACTORS-ASPHALT AND CONCRETE with score 92
LOG: Not matched, TLANNERS  CURRIER and TANNERS  GURRIERS with score 88
LOG: Not matched, ENGINEERSDCONSULTLNG and ENGINEERS-C(WNSULTLNG with score 88
LOG: Not matched, BRUSH AND BROOM DEALERS-F WHOLESALE and HAT DEALERS-WHOLESALE with score 71
LOG: Not matched, GAB AND ELECTRIC COMPANIES and ELECTRIC COMPANIES with score 82
LOG: Not matched, JEWELERS' MFG-COSTUME and JEWELRY MFRS-COSTUME with score 83
LOG: Not matched, PADVERTISI

LOG: Not matched, I'IRE EXTINGUISHER-SALES  SERVICE and 'EXTINGUISHERS-SALES  RECHARGING with score 74
LOG: Not matched, 'AUTOMOBILE BODY AND FENDER WORKS and AUTOMOBILE BODY WORKS with score 78
LOG: Not matched, WATCHES, CHOCKS  JEWELLRY- WHOLESALE and PAINTS, OILS  VARNISH DEALERS- WHOLESALE with score 61
LOG: Not matched, FFINSURANCE-INLAND MARINE and INSURANCE-OCEAN MARINE with score 81
LOG: Not matched, HOU'SEWARES and  ROSARIES with score 60
LOG: Not matched, OIL BUMING  L'UMACES and I6OIL BURNING FURNACES with score 71
LOG: Not matched, LAND DEVELOPERS and LAND SURVEYORS with score 69
LOG: Matched! F'FIREPLACE FURNISHINGS and FIREPLACE FURNISHINGS with score 95
LOG: Matched! PAINTS. OILS  VARNISH DEALERS- WHOLESALE and PAINTS, OILS  VARNISH DEALERS- WHOLESALE with score 98
LOG: Matched! LINENS-RETAIL  LINOTYPE  MONOTYPE COM- POSITIO and LLNENS-RETAIL  LINOTYPE  MONOTYPE COM- POSITIO with score 98
LOG: Matched! PACKING-STEAM, WATER AND and PACKING-STEAM, WATER ANDAIR with score 9

...

LOG: Not matched, PSYCHOLOGISTS and ASZH HOISTS with score 58
LOG: Not matched, ENGRAVERS-PHOTO and E NGRAVERS- CARD with score 65
LOG: Not matched, SOCIAL SERVICE AGENCIES and AIR LINE TICKET AGENCIES with score 64
LOG: Matched! INSURANCE COMPANIES-LIFE and INSURANCE COMPANIES-LIIE with score 96
LOG: Not matched, WESTMINSTER and WINER with score 63
LOG: Not matched, JEWELRY SOLDERERS INSIGNIA SPECIALISTS and JEWELRY MANUFACTURERS METAL SPE CIALTIES with score 62
LOG: Not matched, A FUNERAL DIRECTORS  and PUNERAL DIRECTORS' SERVICE with score 74
LOG: Matched! PUBLIC RELATIONS COUNSEL and PUHLIC RELATIONS COUNSEL with score 96
LOG: Not matched, PLASTIC PRODUCTS-MFRS and GLASS PRODUCTS with score 69
LOG: Not matched, SCHOOLS  ACADEMIES and COLLECTIONS  GOLLEGOS, SCHOOLS  ACADEMIES with score 61
LOG: Not matched, TRAVEL BUREAUS and TRAFFIC BUREAUS  MGRS with score 63
LOG: Not matched, ORGANIZATIONS - BUSINESS. CIVIC AND PROFESSIONAL and ORGANIZATIONS- YOUNG PEOPLES with score 58
LOG: Not 

LOG: Matched! CONTRACTORS -CARPENTER and GONTRACTORS-CARPENTER with score 93
LOG: Not matched, BOOKSELLERS and OLOCKS-DEALERS with score 72
LOG: Not matched, FURNITURE DEALERS-SECOND HAND and ITFURNITURE DEALERS-OFFICE FURNI TURE with score 64
LOG: Not matched, HOMES  ASYLUMS and H.OMES INSURED with score 57
LOG: Not matched, SCREW MACHINE PRODUCTS and C LEANING PRODUCTS with score 70
LOG: Not matched, BILLIARD  POCKET BILLIARD ROOMS and BILLIARD ROOMS SUPPLIES with score 52
LOG: Not matched, HARDWARE DEALERS-RETALL and SAIE DEALERS-RETAIL with score 76
LOG: Not matched, LAW OFFICB and LAWN SERVICE with score 55
LOG: Not matched, CURTAINS  DRAPERIES and TANKS  TAWERS with score 63
LOG: Not matched, BOWLING ALLEYS  and BOWLING ALLEY BUILDERS with score 76
LOG: Not matched, ART GALLERIES and BOAT BATTERIES with score 67
LOG: Not matched, JEWELERS BINDINGS and JEWELERS  CARDS with score 69
LOG: Not matched, PLORISTS-RETALL and PLASTICS-RETAIL with score 73
LOG: Not matched, SAND  GRAVEL D

LOG: Not matched, CHIROPRACTIC PHYSICIANS and CIOMMERCIAL SIGNS with score 55
LOG: Not matched, PLUMBING SUPPLIES-DEALERS and PLUMBING SUPPLIES-WHOLESALE with score 85
LOG: Not matched, PAINT, OIL  VARNISH DEALERS-RETAIL and 'PAINT, OIL  VARNISH DEALERS- WHOLESALE with score 85
LOG: Not matched, CABINET MAKERS and CARNET CLEANERS with score 69
LOG: Not matched, HEATING APPARATUS  APPLIANCES and LIFE SAVING APPARATUS AND SUPPLI with score 69
LOG: Not matched, THEATRES and TEXTILES with score 63
LOG: Not matched, JUNK DEALERS and PENCE DEALERS with score 72
LOG: Not matched, CLERGY and CLERGYMEN-  with score 71
LOG: Not matched, MEATS--RETAIL and SHOES-RETAIL with score 72
LOG: Not matched, CITY DIRECTORY PUBLISHERS and L'DIRECTORY PUBLISHERS with score 85
LOG: Not matched, MASSAGE and GAS RANGES with score 59
LOG: Not matched, MANUFACTURERS AGENTS and MANUFACTURERSL SUPPLIES with score 74
LOG: Not matched, NEWS DEALERS  and HARNESS DEALERS with score 79
LOG: Not matched, SCHOOLS-PUBLIC 

LOG: Not matched, SALARY and I.OANS-SALARY with score 63
LOG: Not matched, FLORISTS-RETAIL and PLASTICS-RETAIL with score 73
LOG: Not matched, FISH DEALERS-RETAIL and SAIE DEALERS-RETAIL with score 84
LOG: Not matched, TRUCKING and MOTOFR TRUCKING with score 70
LOG: Not matched, SUPPLIES-DEALERS and SAFES-DEALERS with score 76
LOG: Matched! CLEANERS-GARMENTS, CURTAINS  DRAPERIES and GLEANSERS-GARMENTS, CURTAINS AND DRAPERIES with score 93
LOG: Not matched, CONTRACTORS-PLUMBING and CONTRACTORS-PLUGGING with score 90
LOG: Not matched, OFFICE SUPPLIES  EQUIPMENT and PRINTERS SUPPLIES AND EQUIPMENT with score 77
LOG: Not matched, JEWELERS and JREWELRY with score 75
LOG: Not matched, NURSES-GRADUATE and PRINTERS-GRAVURE with score 58
LOG: Not matched, FURNISHED ROOMS and FUNDRAISERS with score 62
LOG: Not matched, APARTMENTS AND TOWN HOUSE and FRATERNITY AND SORORITY HOUSH with score 59
LOG: Not matched, ICE DEALERS and PENCE DEALERS with score 83
LOG: Not matched, FURNITURE DEALERS-RETALL 

LOG: Not matched, DENTISTS and DIETITIAHS with score 67
LOG: Not matched, MUSIC TEACHERS and MUSIC STORES with score 69
LOG: Not matched, NURSES and NASELS with score 67
LOG: Not matched, LAW BOOKS, LAWYERS and LAW BOOKS  LAWN SEEDS with score 77
LOG: Not matched, LAW OFFICES and LAWN SERVICE with score 61
LOG: Not matched, SHOE REPAIRERS and SHOE REPAIR with score 88
LOG: Not matched, PHYSICIANS  SURGEONS and PHYSICIANS'  SURGEONS' BX- CHANGE with score 75
LOG: Not matched, LIQUORS-RETAIL and LIQUORS DEALERS- RETAIL with score 76
LOG: Not matched, RESTAURANTS  LUNCH ROOMS and RESTAURANTS-FAST FOOD with score 62
LOG: Not matched, BARBERS and BERS with score 73
LOG: Not matched, VARIETY STORES and PACKAGE STORES with score 64
LOG: Not matched, LODGING HOUSES and LODUES with score 60


In [8]:
pkl.dump(map_dict, open('trueheaders_match_dict.pkl', 'wb'))

In [17]:
def assign_matched(D):
    matched = []
    for h in D.true_headers:
        if h in map_dict: 
            matched.append(map_dict[h][1])
        else: 
            print("Known: ", h, " not in map_dict")
            matched.append(h)
    return matched
def assign_score(D):
    scores = []
    for h in D.true_headers:
        if h in map_dict: 
            scores.append(map_dict[h][0])
        else:
            print("Known: ", h, " not in map_dict")
            scores.append(np.nan)
    return scores
def assign_bool(D):
    is_matched = []
    for h in D.true_headers:
        if h in map_dict:
            is_matched.append(map_dict[h][2])
        else:
            print("Known: ", h, " not in map_dict")
            is_matched.append("KNOWN")
    return is_matched
df = df.assign(matched = assign_matched, score = assign_score, is_matched = assign_bool).sort_values("count")

('Known: ', 'SHOE DEALERS-MEWS  BOYS, -RETAIL', ' not in map_dict')
('Known: ', 'BOOKS, NEWSPAPERS  PERIODI- CALS', ' not in map_dict')
('Known: ', 'SHJPPING TAG MFRS', ' not in map_dict')
('Known: ', 'MANUFACTURING SPAEE-RENTALS', ' not in map_dict')
('Known: ', 'PLOOR RESURFACING MACHINES- RENTED', ' not in map_dict')
('Known: ', 'PRINTERS-GRAVURE', ' not in map_dict')
('Known: ', 'QREES-SPRAYING', ' not in map_dict')
('Known: ', 'SMELTER-GOLD, SILVER  PLATINUM', ' not in map_dict')
('Known: ', 'LINOLEUM AND TILE INLAID', ' not in map_dict')
('Known: ', "ICHILDREN'S--INFANTS' WEAR- RETAIL", ' not in map_dict')
('Known: ', 'TEST BORINGS S', ' not in map_dict')
('Known: ', 'INDIA RUBBER GOODS  HNDUSTRIAL INVESTIGATIOU PAYROLL SERVICE', ' not in map_dict')
('Known: ', 'VALVE MFRS', ' not in map_dict')
('Known: ', 'PRINTERS, ROLLERS-MFRS', ' not in map_dict')
('Known: ', 'REFRIGERATORS-E LECTRIC', ' not in map_dict')
('Known: ', 'BLIP COVER MANUFACTURERS', ' not in map_dict')
('Known: ',

('Known: ', 'WEAVING-CLOTHING', ' not in map_dict')
('Known: ', 'POLDING BOX MANUFACTURERS', ' not in map_dict')
('Known: ', 'SHIPPING TAN MFRS', ' not in map_dict')
('Known: ', 'ROOFING MATERIALS  ASPHALT A SHINGLES', ' not in map_dict')
('Known: ', 'ARTIFICIAL FLOWERS-RETAIL', ' not in map_dict')
('Known: ', '6OSTUME JEWELRY', ' not in map_dict')
('Known: ', 'YARN DVERS', ' not in map_dict')
('Known: ', 'NURSES REQISTRV', ' not in map_dict')
('Known: ', "L' MONORAIL SYSTEMS", ' not in map_dict')
('Known: ', 'DISTILLERS  RECTIFIERS,', ' not in map_dict')
('Known: ', 'RAILROADB', ' not in map_dict')
('Known: ', 'HOUSE AND BUILDING MOVERS', ' not in map_dict')
('Known: ', 'OORNIOE MANUFACTURERS', ' not in map_dict')
('Known: ', 'STABLES  STALNLESS STEEL-BARS, SHEETS,', ' not in map_dict')
('Known: ', 'JREWELRY', ' not in map_dict')
('Known: ', 'OF.IICE HMITURE', ' not in map_dict')
('Known: ', "INSURANCE'-MARINE", ' not in map_dict')
('Known: ', "'-'MAILING-LETTERS-ADVERTISINN", ' not i

('Known: ', 'JEWELRY TAU MFRS', ' not in map_dict')
('Known: ', 'PACKING-STEAM, ETC', ' not in map_dict')
('Known: ', 'SODA  MINERAL WATER MFRS  SOLDER  SOLDERING PLUXES-MIRS', ' not in map_dict')
('Known: ', 'PAINTS  GOLORS-ARTISTS', ' not in map_dict')
('Known: ', 'SAMPLE TRUNKS', ' not in map_dict')
('Known: ', 'BURGLAR AND FIRE ALARM NOTIFICATION. WATCHMAN,S SU- PERVI-SORY SERVICE', ' not in map_dict')
('Known: ', "7 'INSNLATLON-PIBERGLAS", ' not in map_dict')
('Known: ', 'ABRASIVE  CUTTING SPECIALISTS', ' not in map_dict')
('Known: ', 'SAIEL DEALERS AND EXPERTS', ' not in map_dict')
('Known: ', "LF'RE'AL ESTATE  INSURANCE", ' not in map_dict')
('Known: ', 'CO-NTI-ACTORS-TRAILER SERVICE', ' not in map_dict')
('Known: ', 'LKPHOTOGRAPHIC EQUIPMENT AND SUPPLIES', ' not in map_dict')
('Known: ', 'SAFETY TANKS-PRLOR', ' not in map_dict')
('Known: ', 'METAL FINDINGS AND SMALL STAMPINGS-MIRS', ' not in map_dict')
('Known: ', "BAKERS' SUPPLIES  EQILIPMENT", ' not in map_dict')
('Known: ', 

('Known: ', ' TREATY MANAGERS INC', ' not in map_dict')
('Known: ', 'WALLS  ASH GANS-GALVANLZED', ' not in map_dict')
('Known: ', 'COTTON  WOOLEN WASTE  COUNSELORS-AT-LAW', ' not in map_dict')
('Known: ', 'VENTILATING EQUIPMENT', ' not in map_dict')
('Known: ', ' RE FRIGE RATION', ' not in map_dict')
('Known: ', 'SPORTING GOODS-RETAIL- WHOLESALE', ' not in map_dict')
('Known: ', 'ADDRESSING MACHINES ADDRESSOGRAPH-MULTIGRAPH CORP', ' not in map_dict')
('Known: ', 'ENGINEERS-AIR CONDITIONING AND HEATING', ' not in map_dict')
('Known: ', 'STORM. WINDOWS AND DOORS-- ALUMINUM', ' not in map_dict')
('Known: ', 'CONSTABLE S', ' not in map_dict')
('Known: ', 'FREE ZERS-HOME FREE ZERS', ' not in map_dict')
('Known: ', "I'LOOR SANDING", ' not in map_dict')
('Known: ', "I'URNLTURE PACKERS  MOVERS", ' not in map_dict')
('Known: ', 'RADIO SETS  PARTS-MTRS', ' not in map_dict')
('Known: ', 'NO SOLICITING CONTRIBUTIONS PLEASE', ' not in map_dict')
('Known: ', "LKWINES'  LIQUO-RS-IMPORTERS", ' not in 

In [18]:
known = df.loc[df.matched != "no_header"].reset_index(drop=True).drop_duplicates().sort_values("count")
unknown = df.loc[df.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers").sort_values("count")

In [11]:
# def internal_match(headers, true_headers, map_dict):
#     # Matches ocr headers to true headers
#     for header in headers:
#         true_headers = [h for h in true_headers if h != header]
#         if header not in map_dict:
#             score_list = [(score(header, true_header), true_header) for true_header in true_headers]
#             sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
#             score_tuple = sorted_score[0]
#             if score_tuple[0] > 90:
#                 print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = score_tuple
#             else:
#                 print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
#                 map_dict[header] = (score_tuple[0], "no_header")
#     return map_dict
# def assign_matched_internal(D):
#     matched = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             matched.append(internal_dict[h][1])
#         else: 
#             matched.append(h)
#     return matched
# def assign_score_internal(D):
#     scores = []
#     for h in D.true_headers:
#         if h in internal_dict: 
#             scores.append(internal_dict[h][0])
#         else:
#             scores.append(np.nan)
#     return scores

# internal_dict = {}
# internal_dict = internal_match(list(unknown.true_headers), list(unknown.true_headers), internal_dict)

# internal_matched = unknown
# internal_matched = internal_matched.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown2 = internal_matched.loc[internal_matched.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# internal_dict = internal_match(list(unknown2.true_headers), list(unknown2.true_headers), internal_dict)

# internal_matched2 = unknown2
# internal_matched2 = internal_matched2.assign(matched = assign_matched_internal, score = assign_score_internal)
# unknown3 = internal_matched2.loc[internal_matched2.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers")

# unknown3

In [12]:
def calculate_scores(df):
    l = len(df.true_headers)
    score_matrix = np.zeros((l,l))
    i = 0
    for h in df.true_headers:
        score_matrix[i, :] = df.true_headers.apply(score, args=(h,)).values
        print("Row number: {} of {}".format(i, l - 1))
        i += 1
    return score_matrix

def remove_repeat(df, scores):
    prelist = list(df.true_headers)
    i = 0
    for h in df.true_headers:
        j = 0
        for removal in scores[i, :]:
            if removal and j > i:
                try:
                    df = df[df["true_headers"] != prelist[j]]
                    print("LOG: " + prelist[j] + " deleted!")
                except KeyError:
                    print("LOG: " + prelist[j] + " Already deleted")
            j += 1
        i += 1
    return df

In [13]:
scores = calculate_scores(unknown)

Row number: 0 of 7611
Row number: 1 of 7611
Row number: 2 of 7611
Row number: 3 of 7611
Row number: 4 of 7611
Row number: 5 of 7611
Row number: 6 of 7611
Row number: 7 of 7611
Row number: 8 of 7611
Row number: 9 of 7611
Row number: 10 of 7611
Row number: 11 of 7611
Row number: 12 of 7611
Row number: 13 of 7611
Row number: 14 of 7611
Row number: 15 of 7611
Row number: 16 of 7611
Row number: 17 of 7611
Row number: 18 of 7611
Row number: 19 of 7611
Row number: 20 of 7611
Row number: 21 of 7611
Row number: 22 of 7611
Row number: 23 of 7611
Row number: 24 of 7611
Row number: 25 of 7611
Row number: 26 of 7611
Row number: 27 of 7611
Row number: 28 of 7611
Row number: 29 of 7611
Row number: 30 of 7611
Row number: 31 of 7611
Row number: 32 of 7611
Row number: 33 of 7611
Row number: 34 of 7611
Row number: 35 of 7611
Row number: 36 of 7611
Row number: 37 of 7611
Row number: 38 of 7611
Row number: 39 of 7611
Row number: 40 of 7611
Row number: 41 of 7611
Row number: 42 of 7611
Row number: 43 of 761

Row number: 348 of 7611
Row number: 349 of 7611
Row number: 350 of 7611
Row number: 351 of 7611
Row number: 352 of 7611
Row number: 353 of 7611
Row number: 354 of 7611
Row number: 355 of 7611
Row number: 356 of 7611
Row number: 357 of 7611
Row number: 358 of 7611
Row number: 359 of 7611
Row number: 360 of 7611
Row number: 361 of 7611
Row number: 362 of 7611
Row number: 363 of 7611
Row number: 364 of 7611
Row number: 365 of 7611
Row number: 366 of 7611
Row number: 367 of 7611
Row number: 368 of 7611
Row number: 369 of 7611
Row number: 370 of 7611
Row number: 371 of 7611
Row number: 372 of 7611
Row number: 373 of 7611
Row number: 374 of 7611
Row number: 375 of 7611
Row number: 376 of 7611
Row number: 377 of 7611
Row number: 378 of 7611
Row number: 379 of 7611
Row number: 380 of 7611
Row number: 381 of 7611
Row number: 382 of 7611
Row number: 383 of 7611
Row number: 384 of 7611
Row number: 385 of 7611
Row number: 386 of 7611
Row number: 387 of 7611
Row number: 388 of 7611
Row number: 389 

Row number: 699 of 7611
Row number: 700 of 7611
Row number: 701 of 7611
Row number: 702 of 7611
Row number: 703 of 7611
Row number: 704 of 7611
Row number: 705 of 7611
Row number: 706 of 7611
Row number: 707 of 7611
Row number: 708 of 7611
Row number: 709 of 7611
Row number: 710 of 7611
Row number: 711 of 7611
Row number: 712 of 7611
Row number: 713 of 7611
Row number: 714 of 7611
Row number: 715 of 7611
Row number: 716 of 7611
Row number: 717 of 7611
Row number: 718 of 7611
Row number: 719 of 7611
Row number: 720 of 7611
Row number: 721 of 7611
Row number: 722 of 7611
Row number: 723 of 7611
Row number: 724 of 7611
Row number: 725 of 7611
Row number: 726 of 7611
Row number: 727 of 7611
Row number: 728 of 7611
Row number: 729 of 7611
Row number: 730 of 7611
Row number: 731 of 7611
Row number: 732 of 7611
Row number: 733 of 7611
Row number: 734 of 7611
Row number: 735 of 7611
Row number: 736 of 7611
Row number: 737 of 7611
Row number: 738 of 7611
Row number: 739 of 7611
Row number: 740 

Row number: 1044 of 7611
Row number: 1045 of 7611
Row number: 1046 of 7611
Row number: 1047 of 7611
Row number: 1048 of 7611
Row number: 1049 of 7611
Row number: 1050 of 7611
Row number: 1051 of 7611
Row number: 1052 of 7611
Row number: 1053 of 7611
Row number: 1054 of 7611
Row number: 1055 of 7611
Row number: 1056 of 7611
Row number: 1057 of 7611
Row number: 1058 of 7611
Row number: 1059 of 7611
Row number: 1060 of 7611
Row number: 1061 of 7611
Row number: 1062 of 7611
Row number: 1063 of 7611
Row number: 1064 of 7611
Row number: 1065 of 7611
Row number: 1066 of 7611
Row number: 1067 of 7611
Row number: 1068 of 7611
Row number: 1069 of 7611
Row number: 1070 of 7611
Row number: 1071 of 7611
Row number: 1072 of 7611
Row number: 1073 of 7611
Row number: 1074 of 7611
Row number: 1075 of 7611
Row number: 1076 of 7611
Row number: 1077 of 7611
Row number: 1078 of 7611
Row number: 1079 of 7611
Row number: 1080 of 7611
Row number: 1081 of 7611
Row number: 1082 of 7611
Row number: 1083 of 7611


Row number: 1374 of 7611
Row number: 1375 of 7611
Row number: 1376 of 7611
Row number: 1377 of 7611
Row number: 1378 of 7611
Row number: 1379 of 7611
Row number: 1380 of 7611
Row number: 1381 of 7611
Row number: 1382 of 7611
Row number: 1383 of 7611
Row number: 1384 of 7611
Row number: 1385 of 7611
Row number: 1386 of 7611
Row number: 1387 of 7611
Row number: 1388 of 7611
Row number: 1389 of 7611
Row number: 1390 of 7611
Row number: 1391 of 7611
Row number: 1392 of 7611
Row number: 1393 of 7611
Row number: 1394 of 7611
Row number: 1395 of 7611
Row number: 1396 of 7611
Row number: 1397 of 7611
Row number: 1398 of 7611
Row number: 1399 of 7611
Row number: 1400 of 7611
Row number: 1401 of 7611
Row number: 1402 of 7611
Row number: 1403 of 7611
Row number: 1404 of 7611
Row number: 1405 of 7611
Row number: 1406 of 7611
Row number: 1407 of 7611
Row number: 1408 of 7611
Row number: 1409 of 7611
Row number: 1410 of 7611
Row number: 1411 of 7611
Row number: 1412 of 7611
Row number: 1413 of 7611


...

Row number: 6030 of 7611
Row number: 6031 of 7611
Row number: 6032 of 7611
Row number: 6033 of 7611
Row number: 6034 of 7611
Row number: 6035 of 7611
Row number: 6036 of 7611
Row number: 6037 of 7611
Row number: 6038 of 7611
Row number: 6039 of 7611
Row number: 6040 of 7611
Row number: 6041 of 7611
Row number: 6042 of 7611
Row number: 6043 of 7611
Row number: 6044 of 7611
Row number: 6045 of 7611
Row number: 6046 of 7611
Row number: 6047 of 7611
Row number: 6048 of 7611
Row number: 6049 of 7611
Row number: 6050 of 7611
Row number: 6051 of 7611
Row number: 6052 of 7611
Row number: 6053 of 7611
Row number: 6054 of 7611
Row number: 6055 of 7611
Row number: 6056 of 7611
Row number: 6057 of 7611
Row number: 6058 of 7611
Row number: 6059 of 7611
Row number: 6060 of 7611
Row number: 6061 of 7611
Row number: 6062 of 7611
Row number: 6063 of 7611
Row number: 6064 of 7611
Row number: 6065 of 7611
Row number: 6066 of 7611
Row number: 6067 of 7611
Row number: 6068 of 7611
Row number: 6069 of 7611


Row number: 6364 of 7611
Row number: 6365 of 7611
Row number: 6366 of 7611
Row number: 6367 of 7611
Row number: 6368 of 7611
Row number: 6369 of 7611
Row number: 6370 of 7611
Row number: 6371 of 7611
Row number: 6372 of 7611
Row number: 6373 of 7611
Row number: 6374 of 7611
Row number: 6375 of 7611
Row number: 6376 of 7611
Row number: 6377 of 7611
Row number: 6378 of 7611
Row number: 6379 of 7611
Row number: 6380 of 7611
Row number: 6381 of 7611
Row number: 6382 of 7611
Row number: 6383 of 7611
Row number: 6384 of 7611
Row number: 6385 of 7611
Row number: 6386 of 7611
Row number: 6387 of 7611
Row number: 6388 of 7611
Row number: 6389 of 7611
Row number: 6390 of 7611
Row number: 6391 of 7611
Row number: 6392 of 7611
Row number: 6393 of 7611
Row number: 6394 of 7611
Row number: 6395 of 7611
Row number: 6396 of 7611
Row number: 6397 of 7611
Row number: 6398 of 7611
Row number: 6399 of 7611
Row number: 6400 of 7611
Row number: 6401 of 7611
Row number: 6402 of 7611
Row number: 6403 of 7611


Row number: 6699 of 7611
Row number: 6700 of 7611
Row number: 6701 of 7611
Row number: 6702 of 7611
Row number: 6703 of 7611
Row number: 6704 of 7611
Row number: 6705 of 7611
Row number: 6706 of 7611
Row number: 6707 of 7611
Row number: 6708 of 7611
Row number: 6709 of 7611
Row number: 6710 of 7611
Row number: 6711 of 7611
Row number: 6712 of 7611
Row number: 6713 of 7611
Row number: 6714 of 7611
Row number: 6715 of 7611
Row number: 6716 of 7611
Row number: 6717 of 7611
Row number: 6718 of 7611
Row number: 6719 of 7611
Row number: 6720 of 7611
Row number: 6721 of 7611
Row number: 6722 of 7611
Row number: 6723 of 7611
Row number: 6724 of 7611
Row number: 6725 of 7611
Row number: 6726 of 7611
Row number: 6727 of 7611
Row number: 6728 of 7611
Row number: 6729 of 7611
Row number: 6730 of 7611
Row number: 6731 of 7611
Row number: 6732 of 7611
Row number: 6733 of 7611
Row number: 6734 of 7611
Row number: 6735 of 7611
Row number: 6736 of 7611
Row number: 6737 of 7611
Row number: 6738 of 7611


Row number: 7032 of 7611
Row number: 7033 of 7611
Row number: 7034 of 7611
Row number: 7035 of 7611
Row number: 7036 of 7611
Row number: 7037 of 7611
Row number: 7038 of 7611
Row number: 7039 of 7611
Row number: 7040 of 7611
Row number: 7041 of 7611
Row number: 7042 of 7611
Row number: 7043 of 7611
Row number: 7044 of 7611
Row number: 7045 of 7611
Row number: 7046 of 7611
Row number: 7047 of 7611
Row number: 7048 of 7611
Row number: 7049 of 7611
Row number: 7050 of 7611
Row number: 7051 of 7611
Row number: 7052 of 7611
Row number: 7053 of 7611
Row number: 7054 of 7611
Row number: 7055 of 7611
Row number: 7056 of 7611
Row number: 7057 of 7611
Row number: 7058 of 7611
Row number: 7059 of 7611
Row number: 7060 of 7611
Row number: 7061 of 7611
Row number: 7062 of 7611
Row number: 7063 of 7611
Row number: 7064 of 7611
Row number: 7065 of 7611
Row number: 7066 of 7611
Row number: 7067 of 7611
Row number: 7068 of 7611
Row number: 7069 of 7611
Row number: 7070 of 7611
Row number: 7071 of 7611


Row number: 7365 of 7611
Row number: 7366 of 7611
Row number: 7367 of 7611
Row number: 7368 of 7611
Row number: 7369 of 7611
Row number: 7370 of 7611
Row number: 7371 of 7611
Row number: 7372 of 7611
Row number: 7373 of 7611
Row number: 7374 of 7611
Row number: 7375 of 7611
Row number: 7376 of 7611
Row number: 7377 of 7611
Row number: 7378 of 7611
Row number: 7379 of 7611
Row number: 7380 of 7611
Row number: 7381 of 7611
Row number: 7382 of 7611
Row number: 7383 of 7611
Row number: 7384 of 7611
Row number: 7385 of 7611
Row number: 7386 of 7611
Row number: 7387 of 7611
Row number: 7388 of 7611
Row number: 7389 of 7611
Row number: 7390 of 7611
Row number: 7391 of 7611
Row number: 7392 of 7611
Row number: 7393 of 7611
Row number: 7394 of 7611
Row number: 7395 of 7611
Row number: 7396 of 7611
Row number: 7397 of 7611
Row number: 7398 of 7611
Row number: 7399 of 7611
Row number: 7400 of 7611
Row number: 7401 of 7611
Row number: 7402 of 7611
Row number: 7403 of 7611
Row number: 7404 of 7611


In [19]:
removal_matrix = (scores > 85) & (scores < 100)
internal_matched = remove_repeat(unknown, removal_matrix)
internal_matched = internal_matched.reset_index(drop=True)

LOG: MALLETS-RA.W HIDE deleted!
LOG: CONTRACTORS-PLASTE RING deleted!
LOG: ATITOMOBILE TRIMMERS AND TRIMMINGS deleted!
LOG: PAPER BOXES deleted!
LOG: PLPE deleted!
LOG: LMILEANSERS  DYERS deleted!
LOG: ADVERTISING NOVELTIES deleted!
LOG: SUBURBAN HOMES deleted!
LOG: WATCH CASE MFRS deleted!
LOG: HARRISON ORGANIZATIONS-WELFARE deleted!
LOG: E'L'ECT.ROPLAT'GNG SUPPLIES AND LIQ UMMENT deleted!
LOG: TRIBUTORS deleted!
LOG: PIPE FITTINGS, HANGERS, VALVES  SPECIALTIES deleted!
LOG: PHYSICIANS AND HOSPITAL SUPPLIES- MFRS deleted!
LOG: ESTATE PLANNING AND MANAGEMENT deleted!
LOG: CONFE CTIONERY-W HOLESALE deleted!
LOG: H'OLLOW METAL DOORS AND FRAMES deleted!
LOG:  HE ATING INSTALLATION deleted!
LOG: J EWE LRY-MANUFA CTURERS-  deleted!
LOG: INSURANCE-TRANSPORTATIO deleted!
LOG: INSURANCE COMPANJW-AUTO- MOBILE deleted!
LOG: CABINETS  COUNTERS deleted!
LOG: TINSMITHS  deleted!
LOG: BRONZE AND BRASS PRODUCTS- MFRS deleted!
LOG: PAWNBROKE R S deleted!
LOG: YARNS -MAN UFAC TURERS deleted!
LOG: C ASH

LOG: MANNER MANUFACTURERS deleted!
LOG: RUING CABINETS  SUPPLIES deleted!
LOG: SLIPPER MFRS deleted!
LOG: NURSING HOMES- HALLWORTH HOUSE EPISCOPAL deleted!
LOG: MIMEOGRAPH MACHINES deleted!
LOG: ENGINEERS-GHEMICAL deleted!
LOG: WOTTEE ROASTERS-WHOLESALE deleted!
LOG: ANGELL deleted!
LOG: ALUMINUM WARE -MANUFAC - TURERS deleted!
LOG: EXEAVATING CONTRACTORS deleted!
LOG: SHOE MFRS GOODS AND SUPPLIES deleted!
LOG: CONTRACTORS-BUILDING- G ENERAL deleted!
LOG: TYPESETTING deleted!
LOG: PHONOGRAPHS  RECORDS-DEALERS deleted!
LOG: STEAMBOAT LINES  deleted!
LOG: OIL PRODUCERS AND SHIPPERS deleted!
LOG: DICTATING MACHINES deleted!
LOG: FLORIDA OFFICE deleted!
LOG: WALLBOARD-MASONITE HOMASOTE- ARMSTRONG-NU-WOOD- BARCLAY deleted!
LOG: SATE MFRS deleted!
LOG: LLBUII..DING PAINTING deleted!
LOG:  CHAIN MFRS deleted!
LOG: INSURANCE INVESTIGATORSL deleted!
LOG: ACCOUNTANTS-REGISTERED PUBLIC deleted!
LOG: MERCERLZLNG MACHINERY deleted!
LOG: WEAVERS  R-E-WEAVEM-GAR- MENT deleted!
LOG: CARPETS  RUGS  '-'

LOG: CHEMISTS-MANUFAC TURING deleted!
LOG: MACHINERY DEALERS-USED deleted!
LOG: LADIEQ TAILORS  deleted!
LOG: AND WOOL deleted!
LOG: BROKERS-STOCKS AND' BONDS deleted!
LOG: ALCOHOL DEALERS deleted!
LOG: LOCKSMITHS  deleted!
LOG: APARTMENTS deleted!
LOG: REPAIR. SHOPS deleted!
LOG: PWTAGE STAMPS AND PHILATEEZ PHILATE LIC SUPP LIES  deleted!
LOG: BANK BOOKS deleted!
LOG: MUTUAL FUNDS deleted!
LOG: AUTOMOBILE AGENCIES-PASSEN- GER' CARS deleted!
LOG: CORDAGE AND TWINE- MFRS AND WHOL deleted!
LOG: TRANSLATORS AND INTERPRETERS deleted!
LOG: SAND  GRAVEL DEALEM deleted!
LOG: BOWLING LAN, deleted!
LOG: GAGES deleted!
LOG: ROVLNG CANS deleted!
LOG: HERBS AND ROOTS deleted!
LOG: CELLOPHANE GOODS-MTRS deleted!
LOG: ELECTRIC NEEDLE TREATMENT  ELECTRIC RAILWAYS deleted!
LOG: BLESCTRIC IRONERS deleted!
LOG: BOX MIRS--WOODE deleted!
LOG: DELIVERY CARS deleted!
LOG: SHINGLES deleted!
LOG:  AWNINGS AND TE NTS deleted!
LOG: GENERAL MERCHANDISE- USED deleted!
LOG: CONTRACTORS-ELECTRICAL-  deleted!
LOG: R

LOG: DRUGGISTS deleted!
LOG: ELECTRIC MOTOR GENERATORS deleted!
LOG: NUTS, BOLTS C  OCHE BUILDINGS deleted!
LOG: ESTATES-OTTLCES OF  deleted!
LOG: PUBLIC ADDRESS AND SOUND EQUIPMENT deleted!
LOG: VENETIAN BLINDFMH-S deleted!
LOG: ENGLNEERS-MECHANLCAL deleted!
LOG: DISPLAY FORMS  FIXTURES deleted!
LOG: PHOTO ENGRAVERS  deleted!
LOG: SCRAN IRON  deleted!
LOG: WUOD ENGRAVERS  deleted!
LOG: FENCE MANUFACTURERS deleted!
LOG: HAT AND CAP DEALERS-RETAIL  deleted!
LOG: CANSTLC SODA MFRS deleted!
LOG: WEDDING BOUQUETS deleted!
LOG: DRESS PLAITING AND TUCKING deleted!
LOG: WAREHOUSES-MERCHANDISE deleted!
LOG: TATTOOING deleted!
LOG: SCHOOLS  ACADEMIES deleted!
LOG: AIR LME COMPANIES deleted!
LOG: EYESIGHT SPECIALISTS deleted!
LOG: BEARINGS -BALL AND ROLLER- MANUFACTURERS deleted!
LOG: TILE--I'LOOR, WALL AND FIREPLACE deleted!
LOG: REFRIGERATORS-SAIE AND SERVICE deleted!
LOG: FOUNDERS-IRO deleted!
LOG: REPAIR SHOPS deleted!
LOG: CABLE TELEVISION deleted!
LOG: PUBLIC ADDRESS AND SOUND EQUIPMENT de

LOG: MOVERS deleted!
LOG: PHYSICIANS AND SURGEONS  deleted!
LOG: ROVLNG CANS deleted!
LOG: CELLOPHANE GOODS-MTRS deleted!
LOG: TNIST COMPANIES  deleted!
LOG: ELECTRIC NEEDLE TREATMENT  ELECTRIC RAILWAYS deleted!
LOG: ALCOHOL deleted!
LOG: AUTOMOBILE GARAG ES deleted!
LOG: MACHINERY ; MFRS deleted!
LOG: FAMILY LAUNDRY deleted!
LOG: BOILER B; RADIATOR DEALERS- STEAM  WATER , deleted!
LOG: EOUSE PUMISHING GOODS- WHOLESALE deleted!
LOG: RADIO SETS AND SUPPLIES- WHOLESALE deleted!
LOG: SODA FOUNTAIN SUPPLIES deleted!
LOG: RADIATM' REPAIRERS-AUTOMOBILE deleted!
LOG: SOUND RECORDING SYSTEMS deleted!
LOG: SAVINGS AND LOAN ASSOCIATIONS deleted!
LOG:  DEALERS deleted!
LOG: TRUCKS-USED deleted!
LOG: CONFECTIONESRY-WHOLESALE deleted!
LOG: MOTOR SCOOTERS deleted!
LOG: BOOK SHOPS deleted!
LOG: PLIES-DEALERS deleted!
LOG: BICYCLE SALES AND. SERVICE deleted!
LOG: MOTORCYC LE DEALERS deleted!
LOG: SCALE MFRS deleted!
LOG: CHARTERED BUS SERVICE deleted!
LOG: MALT AND HOPS deleted!
LOG: SHOPPING SERVICE 

...

LOG: WASHING MACHINES-DEALERS deleted!
LOG: INSURANCE COMPANIES-AUTOMOBILE deleted!
LOG: SCHOOLS AND COLLEGES- JOHNSON  WALES UNIVERSITY CENTER FOR CONTINUING deleted!
LOG: VENETIAN BLINDS deleted!
LOG:  NEWS DEALERS deleted!
LOG: PHOTOGRAPHERS-COMMERCIAL deleted!
LOG: ICE CREAM DEALERS deleted!
LOG: DELICAT ESSEN- RETAIL deleted!
LOG: WATCH CRYSTAL MFRS WATCH  JEWELRY REPAIRERS  deleted!
LOG: STOVES  RANGES-RETALL deleted!
LOG: TEXTILE MIRS-WOOLEN WORSTED GOODS deleted!
LOG: VIDEO MOVIES-SALES AND RENTALS deleted!
LOG: CHAIN MANUFACTURERS deleted!
LOG: MONUMENTS, MARKERS AND STATUARY deleted!
LOG: WATCHCLOCK AND JEWELRY REPAIRING deleted!
LOG: CONTRACTORS-MASOM deleted!
LOG: BROKERS-BUSINESS deleted!
LOG: FUNERAL DIRECTORS' SUPPLIES deleted!
LOG: GROCERII'E AND MEATS- RETAIL  deleted!
LOG: ELECTRICAL COUTRACTORS deleted!
LOG: CHEMICALS-MFRS deleted!
LOG: CLOTHING DEALERS-MENS  BOY-RETAIL deleted!
LOG: BARBERS' SUPPLIES deleted!
LOG: RADIO REPAIRING deleted!
LOG: MANUFACTURERS REPRESEN

LOG: GLASS DEALERS-WINDOW AND PLATE deleted!
LOG: CLUBS-ATHLETIC deleted!
LOG: ORGANIZATIONS- PATRIOTIC AND VETERAN S deleted!
LOG: BOX MIRS-PAPER deleted!
LOG: RESTAURANT deleted!
LOG: CHURCH GOODS deleted!
LOG: VENTILATOR MFRS deleted!
LOG: PLASTIC PRODUCTS- MANUFACTURERS deleted!
LOG: CHALKSTONE  deleted!
LOG: ASSOCIATIONS  CLUBS-COMMERCIAL deleted!
LOG: MUSIC AND MUSICAL INSTRUMENT DEALERS deleted!
LOG: FOOD PRODUCTS MIRS-PREPARED deleted!
LOG: BANKS  TRUST COMPANIES  deleted!
LOG: REGALIA MFRS deleted!
LOG: METAL GOODS-MIRS deleted!
LOG: CLOTHING DEALERS-SEEOND HAND deleted!
LOG: OILS AND LUBRICANTS deleted!
LOG: BOOKS-RETAIL deleted!
LOG: CENTREDALE deleted!
LOG: TRUST COMPANIES  deleted!
LOG: CARPET AND RUG CLEANERS deleted!
LOG: WATCH CRYSTAL MIN WATCH  JEWELRY REPAIRERS deleted!
LOG: JEWELERS' SUPPLIES deleted!
LOG: SHOE SHLNERS deleted!
LOG: MATTRESS MFRS  deleted!
LOG: AUTOMOBILE ACCESSORIES  PARTS-RETALL deleted!
LOG: AUTOMOBILE ACCESSORIES  PARTS -RETAIL deleted!
LOG: GOLD

LOG: SOLDER AND SOLDERING FLUX MFRS deleted!
LOG: PHYSICIAN S AND SURGEONS-MD.  deleted!
LOG: HOSPITALS  DISPENSARIES deleted!
LOG: TAILORS  MERCHANT deleted!
LOG: TAILORS-MERCHANT  deleted!
LOG: TAILORS-MERCHANT deleted!
LOG: BUILDING deleted!
LOG: OSTEOPATHIC PHYSICIANS deleted!
LOG: BUSINESS FORMS AND SYSTEMS deleted!
LOG: LAND COMPANIES  deleted!
LOG: BEVERAGES deleted!
LOG: CONTRACTORS-PAINTING AND DECORATING deleted!
LOG: ASSOCIATIONS  CLUBS-COMMERCIAL deleted!
LOG: CATHOLIC CHURCH deleted!
LOG: CLOTHING-USED deleted!
LOG: MEATS-WHOLESALE deleted!
LOG: BANKS  TRUST COMPANIES  deleted!
LOG: BANKS AND TRUST COMPANIES deleted!
LOG: FLORISTS-RETALL deleted!
LOG: CHRISTIAN SCIENCE PRACTITIONERS deleted!
LOG: WOOD CARVERS deleted!
LOG: FRUIT DEALERS-WHOLESALE AND BROKERS deleted!
LOG: ENAMELERS deleted!
LOG: MILL ENGINEERS-MILL SUPPLIES deleted!
LOG: DICTATING MACHINES deleted!
LOG: FLORIDA OFFICE deleted!
LOG: BROKERS-STOCKS AND BONDS deleted!
LOG: LOANS-MORTGAGE deleted!
LOG: STENOGR

LOG: LADIES' TAILORS  deleted!
LOG: CLOTHING DEALERS-MEWS  BOYS-RETAIL deleted!
LOG: BUILDING MATERIALS  SUPPLIES deleted!
LOG: HARDWARE DEALERS -RETAIL deleted!
LOG: ORGANIZATIONS- LABOR deleted!
LOG: BETTER HOMES AND GARDENS deleted!
LOG: CONFECTIONERY  ICE CREAM- RETAIL  deleted!
LOG: INSURANCE deleted!
LOG: AUTOMOBILE DEALERS-PASSENGER CARS deleted!
LOG: OILS  LUBRICANTS-DEALERS deleted!
LOG: ASSAYERS  RETINARS ASSOCIATIONS  CLUBS-CTNNMER- CLAL deleted!
LOG: YARNS-MFRS  WHOLESALE deleted!
LOG: WATCH CRYSTAL MFRS WATCH AND JEWELRY REPAIRERS deleted!
LOG: FLORISTS-RETAIL deleted!
LOG: PHYSICIAN S AND SURGEONS-MD.  deleted!
LOG: WATCH. CLOCK AND JEWELRY REPAIRING deleted!
LOG: ASSAYERS  REIINERS ASSOCIATIONS  CLUBS deleted!
LOG: POULTRY DEALERS-RETALL deleted!
LOG: WHOLESALE deleted!
LOG: CONTRACTORS-ELECTRICAL deleted!
LOG: CONTRACTORS- ELECTRICAL deleted!
LOG: CLOTHING DEALERS-WOMEN'S  MISSES-RETAIL deleted!
LOG: ENGRAVERS-PHOTO deleted!
LOG: CASTING MFRS deleted!
LOG: CONTRACTORS- 

LOG: PHYSICIANS AND SURGEONS-M D deleted!
LOG: PHYSICIANS AND SURGEONS-M.D deleted!
LOG: PHYSICIANS AND SURGEONS deleted!
LOG: LIQUOR AND WLNES-RETAIL deleted!
LOG: NOVELTIES-MIRS deleted!
LOG: BUILDINGS-OTFICE  PUBLIC deleted!
LOG: BUILDINGS-OFIICE  PUBLIC deleted!
LOG: BULLDINGS-OFFICE  PUBLIC deleted!
LOG: BUILDINQS-OFFICE AND PUBLIC deleted!
LOG: BUILDINGS-OFFLCE  PUBLIC deleted!
LOG: BUILDINGS-OTFICE AND PUBLIC deleted!
LOG: BUILDINGS-OFFICE  PUBLIC deleted!
LOG: BUILDINGS-OFFICE AND PUBLIC deleted!
LOG: GROCERIES AND MEATS- RETAIL deleted!
LOG: GROCERIES AND MEATS-RETAIL deleted!
LOG: CONTRACTERS-BUILDING- GENERAL deleted!
LOG: GONTRACTORS-BUILDING-GENERAL deleted!
LOG: CONTRACTORS-BUILDING- GENERAL  deleted!
LOG: CONTRACTORS-BUILDING-GENERAL deleted!
LOG: PRINTERS-BOOK AND COMMERCIAL deleted!
LOG: PRINTERS-BOOK  COMMERCIAL deleted!
LOG: DELICATESSE deleted!
LOG: KL LODGING HOUSES deleted!
LOG: LODGING HOUSES deleted!
LOG: CLOTHING-RETAIL deleted!
LOG: AUTOMOBILE ACCESSORIES  PAR

In [20]:
internal_matched

,count,true_headers,is_matched,matched,score
0,1.0,"L'MASONS, SUPPLIES",FALSE,no_header,88.0
1,1.0,CATALOGUE PRINTERS,FALSE,no_header,71.0
2,1.0,2TILE BOARD,FALSE,no_header,62.0
3,1.0,WALK DEALERS,FALSE,no_header,74.0
4,1.0,MANS AND MAN PINS,FALSE,no_header,65.0
5,1.0,HOS PITAL CARE ASSOCIATIONS,FALSE,no_header,62.0
6,1.0,ASH DOO'RS ASH DUMPS,FALSE,no_header,87.0
7,1.0,FORGINGS,FALSE,no_header,70.0
8,1.0,MDWARE -BUILDERS,FALSE,no_header,82.0
9,1.0,WALL COVE RINGS -VINYL AND WOOD VENEER,FALSE,no_header,57.0


In [21]:
df

,count,true_headers,is_matched,matched,score
12843,1.0,"SHOE DEALERS-MEWS BOYS, -RETAIL",KNOWN,"SHOE DEALERS-MEWS BOYS, -RETAIL",NaN
9161,1.0,REAL ESTATE BEULAH,KNOWN,REAL ESTATE BEULAH,NaN
9162,1.0,CIRCULAR SLITTING STCS,KNOWN,CIRCULAR SLITTING STCS,NaN
9163,1.0,V HOUN AV,KNOWN,V HOUN AV,NaN
9164,1.0,OONTRACTORS MACHINERY,KNOWN,OONTRACTORS MACHINERY,NaN
9165,1.0,FEATHER DEALERS,KNOWN,FEATHER DEALERS,NaN
9166,1.0,LEATHER GOODS-RETALL,KNOWN,LEATHER GOODS-RETALL,NaN
9167,1.0,CONTRACTORS-BUILDING CLEANING,KNOWN,CONTRACTORS-BUILDING CLEANING,NaN
9168,1.0,SANITARV SUPPLIES,KNOWN,SANITARV SUPPLIES,NaN
9169,1.0,7OOMM; MANUFACTURERS,KNOWN,7OOMM; MANUFACTURERS,NaN
